In [1]:
pip install datasets


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install regex


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install streamlit


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install openai


  Using cached openai-1.35.3-py3-none-any.whl.metadata (21 kB)
Using cached openai-1.35.3-py3-none-any.whl (327 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas numpy matplotlib plotly


Note: you may need to restart the kernel to use updated packages.


# Step 1: Load and Explore the Dataset


In [6]:
# Cell 1: Load the dataset
from datasets import load_dataset

# Load the document-level dataset and take the first 100 samples
ds_document = load_dataset("ccdv/pubmed-summarization", "document")
ds_document = ds_document['train'].select(range(100))

# Load the section-level dataset and take the first 100 samples
ds_section = load_dataset("ccdv/pubmed-summarization", "section")
ds_section = ds_section['train'].select(range(100))

# Check the first sample from each dataset
print(ds_document[0])
print(ds_section[0])


Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

{'article': "a recent systematic analysis showed that in 2011 , 314 ( 296 - 331 ) million children younger than 5 years were mildly , moderately or severely stunted and 258 ( 240 - 274 ) million were mildly , moderately or severely underweight in the developing countries . in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16.2% , 8.6% and 1.5% , for underweight , overweight and obesity , respectively . the prevalence of malnutrition among elementary school aged children in tehran varied from 6% to 16% . anthropometric study of elementary school students in shiraz revealed that 16% of them suffer from malnutrition and low body weight . snack should have 300 - 400 kcal energy and could provide 5 - 10 g of protein / day . nowadays , school nutrition programs are running as the national programs , world - wide . national school lunch program in the united states there are also some reports regarding school feeding programs in developing countries . 

# Step 2: Preprocess the Dataset

In [7]:
# Cell 2: Preprocess the dataset
import re

def preprocess(text):
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text

# Apply preprocessing to document-level dataset
ds_document = ds_document.map(lambda x: {"article": preprocess(x["article"]), "abstract": preprocess(x["abstract"])})

# Apply preprocessing to section-level dataset
ds_section = ds_section.map(lambda x: {"article": preprocess(x["article"]), "abstract": preprocess(x["abstract"])})

# Verify preprocessing
print(ds_document[0])
print(ds_section[0])


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

{'article': 'a recent systematic analysis showed that in 2011 314 296 331 million children younger than 5 years were mildly moderately or severely stunted and 258 240 274 million were mildly moderately or severely underweight in the developing countries in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16 2 8 6 and 1 5 for underweight overweight and obesity respectively the prevalence of malnutrition among elementary school aged children in tehran varied from 6 to 16 anthropometric study of elementary school students in shiraz revealed that 16 of them suffer from malnutrition and low body weight snack should have 300 400 kcal energy and could provide 5 10 g of protein day nowadays school nutrition programs are running as the national programs world wide national school lunch program in the united states there are also some reports regarding school feeding programs in developing countries in vietnam school base program showed an improvement in nu

# Step 3: Define the Summarization Function

In [8]:
# Cell 3: Define a simple extractive summarization function
def extractive_summary(article, num_sentences=3):
    sentences = article.split('. ')
    summary = '. '.join(sentences[:num_sentences])
    return summary

# Test the summarization function
example_article = ds_document[0]['article']
print("Original Article:", example_article)
print("Extractive Summary:", extractive_summary(example_article))


Original Article: a recent systematic analysis showed that in 2011 314 296 331 million children younger than 5 years were mildly moderately or severely stunted and 258 240 274 million were mildly moderately or severely underweight in the developing countries in iran a study among 752 high school girls in sistan and baluchestan showed prevalence of 16 2 8 6 and 1 5 for underweight overweight and obesity respectively the prevalence of malnutrition among elementary school aged children in tehran varied from 6 to 16 anthropometric study of elementary school students in shiraz revealed that 16 of them suffer from malnutrition and low body weight snack should have 300 400 kcal energy and could provide 5 10 g of protein day nowadays school nutrition programs are running as the national programs world wide national school lunch program in the united states there are also some reports regarding school feeding programs in developing countries in vietnam school base program showed an improvement 

In [9]:
# Cell 4: Install required libraries
!pip install datasets regex streamlit openai pandas numpy matplotlib plotly notebook


# Create the Streamlit App:

In [10]:
# Cell 5: Create the Streamlit app (app.py)
import streamlit as st
import openai

# Your OpenAI API key
openai.api_key = "YOUR_API_KEY"

def preprocess(text):
    import re
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    return text

def extractive_summary(article, num_sentences=3):
    sentences = article.split('. ')
    summary = '. '.join(sentences[:num_sentences])
    return summary

def summarize_with_gpt4(article):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Summarize this article: {article}",
        max_tokens=150
    )
    summary = response.choices[0].text.strip()
    return summary

st.title("PubMed Article Summarizer")

uploaded_file = st.file_uploader("Choose a file", type=["txt"])

if uploaded_file is not None:
    article = uploaded_file.read().decode("utf-8")
    st.write("Original Article:", article)

    preprocessed_article = preprocess(article)
    
    # Use extractive summary as a fallback
    summary = extractive_summary(preprocessed_article)
    st.write("Extractive Summary:", summary)
    
    # Use GPT-4 for generative summary
    summary_gpt4 = summarize_with_gpt4(preprocessed_article)
    st.write("GPT-4 Summary:", summary_gpt4)


2024-06-24 21:41:39.724 
  command:

    streamlit run C:\Users\ysoha\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
